# Phase 3 Feedback Workflow

This notebook walks through the ML feedback loop for transformer anomaly detection:
- run the Phase 2 detector + anomaly fusion on a maintenance image
- adjust the proposed bounding boxes with an interactive UI
- persist the feedback to the annotation store and replay dataset
- trigger incremental fine-tuning and inspect evaluation metrics
- export JSON/CSV logs for downstream use.

Run the cells in sequence. Update the configuration cell to pick the image, user ID, and weights.

In [2]:
%pip install opencv-python
%pip install ipywidgets
%pip install scikit-image
%pip install ultralytics


  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Not uninstalling numpy at c:\users\dinuk\appdata\local\programs\python\python311\lib\site-packages, outside environment d:\7th_sem\Software Design Project\transformer-maintenance-system\ano
    Can't uninstall 'numpy'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: d:\7th_sem\Software Design Project\transformer-maintenance-system\ano\Scripts\python.exe -m pip install --upgrade pip


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: d:\7th_sem\Software Design Project\transformer-maintenance-system\ano\Scripts\python.exe -m pip install --upgrade pip


  Using cached scikit_image-0.25.2-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached tifffile-2025.10.16-py3-none-any.whl.metadata (31 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
Using cached scikit_image-0.25.2-cp311-cp311-win_amd64.whl (12.8 MB)
Using cached imageio-2.37.0-py3-none-any.whl (315 kB)
Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
Using cached tifffile-2025.10.16-py3-none-any.whl (231 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: d:\7th_sem\Software Design Project\transformer-maintenance-system\ano\Scripts\python.exe -m pip install --upgrade pip


  Using cached ultralytics-8.3.217-py3-none-any.whl.metadata (37 kB)
  Using cached polars-1.34.0-py3-none-any.whl.metadata (10 kB)
  Using cached ultralytics_thop-2.0.17-py3-none-any.whl.metadata (14 kB)
  Using cached polars_runtime_32-1.34.0-cp39-abi3-win_amd64.whl.metadata (1.5 kB)
Using cached ultralytics-8.3.217-py3-none-any.whl (1.1 MB)
Using cached ultralytics_thop-2.0.17-py3-none-any.whl (28 kB)
Using cached polars-1.34.0-py3-none-any.whl (772 kB)
Using cached polars_runtime_32-1.34.0-cp39-abi3-win_amd64.whl (40.1 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: d:\7th_sem\Software Design Project\transformer-maintenance-system\ano\Scripts\python.exe -m pip install --upgrade pip


In [1]:

from __future__ import annotations

from pathlib import Path
import sys
import json
from typing import Any, Dict, List

import cv2
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output, display

REPO_ROOT = Path.cwd().resolve().parent
if str(REPO_ROOT) not in sys.path:
    sys.path.append(str(REPO_ROOT))
phase2_path = REPO_ROOT / "phase2_fault_type"
if str(phase2_path) not in sys.path:
    sys.path.append(str(phase2_path))

from phase2_fault_type.pipeline.unsupervised import run_unsupervised_pair
from phase2_fault_type.pipeline.detector import run_yolo
from phase2_fault_type.fuse_detections import fuse_regions_with_detections
from phase3_fault_type.annotation_store import AnnotationStore
from phase3_fault_type import config
from phase3_fault_type.feedback_dataset import register_feedback

plt.rcParams["figure.figsize"] = (12, 8)
SESSION_STATE: Dict[str, Any] = {}

In [2]:
transformer_id = "T5"
baseline_idx = 1
maintenance_idx = 1
user_id = "engineer_01"

baseline_path = REPO_ROOT / "Sample_Thermal_Images" / transformer_id / "normal" / f"{transformer_id}_normal_{baseline_idx:03d}.png"
maintenance_path = REPO_ROOT / "Sample_Thermal_Images" / transformer_id / "faulty" / f"{transformer_id}_faulty_{maintenance_idx:03d}.png"
weights_path = REPO_ROOT / "phase2_fault_type" / "weights" / "best.pt"

paths = {
    "baseline": baseline_path,
    "maintenance": maintenance_path,
    "weights": weights_path,
}
for label, path in paths.items():
    status = "OK" if path.exists() else "MISSING"
    print(f"{label:>12}: {path} [{status}]")

SESSION_STATE["transformer_id"] = transformer_id
SESSION_STATE["user_id"] = user_id
SESSION_STATE["baseline_path"] = baseline_path
SESSION_STATE["maintenance_path"] = maintenance_path
SESSION_STATE["weights_path"] = weights_path

    baseline: D:\7th_sem\Software Design Project\transformer-maintenance-system\Sample_Thermal_Images\T5\normal\T5_normal_001.png [OK]
 maintenance: D:\7th_sem\Software Design Project\transformer-maintenance-system\Sample_Thermal_Images\T5\faulty\T5_faulty_001.png [OK]
     weights: D:\7th_sem\Software Design Project\transformer-maintenance-system\phase2_fault_type\weights\best.pt [OK]


In [3]:
def run_phase2_inference(baseline_path: Path, maintenance_path: Path, weights_path: Path) -> Dict[str, Any]:
    work_size = (640, 640)
    config_dict = {
        "processing": {"work_size": list(work_size), "smooth_sigma": 1.0},
        "thresholding": {"mode": "percentile", "value": 2.0, "min_area_pct": 0.002},
        "fuse_weights": [0.6, 0.4],
    }
    result = run_unsupervised_pair(baseline_path, maintenance_path, config_dict)
    detections = run_yolo(weights_path, maintenance_path, conf=0.25, imgsz=work_size[0])
    regions = result.get("anomalies", [])
    fuse_regions_with_detections(regions, detections, iou_thresh=0.7)
    regions = [region for region in regions if region.get("fault_type")]
    result["anomalies"] = regions
    result["detector_summary"] = {
        "weights": str(weights_path),
        "image": str(maintenance_path),
        "params": {"conf_thresh": 0.25, "iou_thresh": 0.7},
        "detections": detections,
    }
    return result

def render_overlay(image_path: Path, annotations: List[Dict[str, Any]]) -> np.ndarray:
    image = cv2.imread(str(image_path))
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    canvas = image.copy()
    for ann in annotations:
        bbox = ann.get("bbox_xyxy") or ann.get("bbox")
        if not bbox:
            continue
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(canvas, (x1, y1), (x2, y2), (0, 200, 0), 2)
        label = ann.get("class_name", "unknown")
        conf_value = ann.get("conf")
        if conf_value is not None:
            label += f" {conf_value:.2f}"
        cv2.putText(canvas, label, (x1, max(0, y1 - 4)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    return cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB)

inference_result = run_phase2_inference(SESSION_STATE["baseline_path"], SESSION_STATE["maintenance_path"], SESSION_STATE["weights_path"])
SESSION_STATE["inference"] = inference_result
SESSION_STATE["image_id"] = Path(inference_result["images"]["maintenance"]).stem

overlay_image = render_overlay(SESSION_STATE["maintenance_path"], inference_result.get("detector_summary", {}).get("detections", []))
display(overlay_image)
print(f"Image ID: {SESSION_STATE['image_id']}")
print(f"Detections: {len(inference_result.get('detector_summary', {}).get('detections', []))}")

array([[[ 34,  34,  34],
        [ 30,  30,  30],
        [ 33,  33,  33],
        ...,
        [ 95,  95,  95],
        [100, 100, 100],
        [100, 100, 100]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [158, 158, 158],
        [163, 163, 163],
        [158, 158, 158]],

       [[  8,   8,   8],
        [  8,   8,   8],
        [ 11,  11,  11],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[  1,   1,   1],
        [  1,   1,   1],
        [  4,   4,   4],
        ...,
        [169, 168, 167],
        [224, 224, 223],
        [249, 249, 249]],

       [[  6,   6,   6],
        [  5,   5,   5],
        [  9,   9,   9],
        ...,
        [250, 250, 249],
        [255, 255, 255],
        [253, 253, 253]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [212, 212, 212],
        [217, 217, 217],
        [209, 208, 209]]

Image ID: T5_faulty_001
Detections: 1


In [4]:
store = AnnotationStore()
SESSION_STATE["store"] = store
class_names = config.get_class_names()

maintenance_image = cv2.imread(str(SESSION_STATE["maintenance_path"]))
if maintenance_image is None:
    raise RuntimeError("Unable to load maintenance image for annotation UI.")
image_height, image_width = maintenance_image.shape[:2]

detections = SESSION_STATE["inference"]["detector_summary"]["detections"]
current_annotations: List[Dict[str, Any]] = []
for idx, det in enumerate(detections):
    current_annotations.append({
        "id": f"det_{idx}",
        "source": "model",
        "class_name": det["class_name"],
        "class_id": det.get("class_id"),
        "bbox_xyxy": [float(v) for v in det["bbox_xyxy"]],
        "conf": det.get("conf"),
    })

annotation_selector = widgets.Dropdown(description="Annotation", options=[], layout=widgets.Layout(width="60%"))
class_selector = widgets.Dropdown(options=class_names, description="Class", layout=widgets.Layout(width="35%"))
x1_field = widgets.FloatText(description="x1", layout=widgets.Layout(width="22%"))
y1_field = widgets.FloatText(description="y1", layout=widgets.Layout(width="22%"))
x2_field = widgets.FloatText(description="x2", layout=widgets.Layout(width="22%"))
y2_field = widgets.FloatText(description="y2", layout=widgets.Layout(width="22%"))
note_field = widgets.Text(description="Action note", layout=widgets.Layout(width="70%"))
general_comment = widgets.Text(description="Final comment", layout=widgets.Layout(width="70%"))
user_field = widgets.Text(value=SESSION_STATE["user_id"], description="User ID", layout=widgets.Layout(width="35%"))

apply_button = widgets.Button(description="Apply", button_style="success")
delete_button = widgets.Button(description="Delete", button_style="danger")
add_button = widgets.Button(description="Add", button_style="info")
save_button = widgets.Button(description="Save & Register", button_style="primary")

status_output = widgets.Output()
overlay_output = widgets.Output()

def clamp_bbox(x1: float, y1: float, x2: float, y2: float) -> List[float]:
    x_min = max(0.0, min(float(image_width - 1), x1))
    x_max = max(0.0, min(float(image_width - 1), x2))
    y_min = max(0.0, min(float(image_height - 1), y1))
    y_max = max(0.0, min(float(image_height - 1), y2))
    if x_max <= x_min:
        x_max = min(float(image_width - 1), x_min + 1.0)
    if y_max <= y_min:
        y_max = min(float(image_height - 1), y_min + 1.0)
    return [x_min, y_min, x_max, y_max]

def refresh_dropdown() -> None:
    options = []
    for ann in current_annotations:
        label = f"[{ann['source']}] {ann['class_name']} ({ann['id']})"
        options.append((label, ann["id"]))
    annotation_selector.options = options
    if options:
        annotation_selector.value = options[0][1]

def populate_fields(annotation_id: str) -> None:
    ann = next((item for item in current_annotations if item["id"] == annotation_id), None)
    if ann is None:
        return
    class_selector.value = ann["class_name"]
    x1_field.value, y1_field.value, x2_field.value, y2_field.value = ann["bbox_xyxy"]

def draw_overlay() -> None:
    with overlay_output:
        clear_output(wait=True)
        vis = maintenance_image.copy()
        for ann in current_annotations:
            x1, y1, x2, y2 = map(int, ann["bbox_xyxy"])
            cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 200, 0), 2)
            cv2.putText(vis, ann["class_name"], (x1, max(0, y1 - 4)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        display(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))

def log_action(action: str, annotation_id: str, bbox_before: List[float] | None, bbox_after: List[float] | None) -> None:
    store.log_action(
        image_id=SESSION_STATE["image_id"],
        transformer_id=SESSION_STATE["transformer_id"],
        action=action,
        user_id=user_field.value,
        bbox_before=bbox_before,
        bbox_after=bbox_after,
        detection_id=annotation_id,
        comment=note_field.value or None,
    )

def handle_selection_change(change: Dict[str, Any]) -> None:
    if change.get("name") == "value" and change.get("new"):
        populate_fields(change["new"])

def handle_apply(_: Any) -> None:
    ann_id = annotation_selector.value
    if not ann_id:
        return
    ann = next((item for item in current_annotations if item["id"] == ann_id), None)
    if ann is None:
        return
    new_bbox = clamp_bbox(x1_field.value, y1_field.value, x2_field.value, y2_field.value)
    bbox_before = list(ann["bbox_xyxy"])
    ann["bbox_xyxy"] = new_bbox
    ann["class_name"] = class_selector.value
    log_action("adjust", ann_id, bbox_before, new_bbox)
    draw_overlay()

def handle_delete(_: Any) -> None:
    ann_id = annotation_selector.value
    if not ann_id:
        return
    ann = next((item for item in current_annotations if item["id"] == ann_id), None)
    if ann is None:
        return
    current_annotations.remove(ann)
    log_action("delete", ann_id, ann["bbox_xyxy"], None)
    refresh_dropdown()
    draw_overlay()

def handle_add(_: Any) -> None:
    new_bbox = clamp_bbox(x1_field.value, y1_field.value, x2_field.value, y2_field.value)
    index = 0
    while True:
        ann_id = f"user_{index:03d}"
        if not any(item["id"] == ann_id for item in current_annotations):
            break
        index += 1
    ann = {
        "id": ann_id,
        "source": "user",
        "class_name": class_selector.value,
        "class_id": None,
        "bbox_xyxy": new_bbox,
        "conf": None,
    }
    current_annotations.append(ann)
    log_action("add", ann_id, None, new_bbox)
    refresh_dropdown()
    annotation_selector.value = ann_id
    draw_overlay()

def handle_save(_: Any) -> None:
    final_annotations = []
    for ann in current_annotations:
        class_name = ann["class_name"]
        class_id = ann.get("class_id")
        if class_id is None and class_name in class_names:
            class_id = class_names.index(class_name)
        final_annotations.append({
            "id": ann["id"],
            "class_name": class_name,
            "class_id": class_id,
            "bbox_xyxy": ann["bbox_xyxy"],
            "source": ann["source"],
            "confidence": ann.get("conf"),
        })
    final_path = store.save_final_state(
        image_id=SESSION_STATE["image_id"],
        maintenance_image_path=SESSION_STATE["maintenance_path"],
        transformer_id=SESSION_STATE["transformer_id"],
        annotations=final_annotations,
        user_id=user_field.value,
        comment=general_comment.value or None,
        metadata={"model_detections": SESSION_STATE["inference"]["detector_summary"]["detections"]},
    )
    register_feedback(
        image_id=SESSION_STATE["image_id"],
        maintenance_image_path=SESSION_STATE["maintenance_path"],
        annotations=final_annotations,
        transformer_id=SESSION_STATE["transformer_id"],
        comment=general_comment.value or None,
        metadata={"saved_from_notebook": True},
    )
    SESSION_STATE["final_annotations"] = final_annotations
    with status_output:
        clear_output(wait=True)
        print(f"Saved final annotations to {final_path}")

annotation_selector.observe(handle_selection_change)
apply_button.on_click(handle_apply)
delete_button.on_click(handle_delete)
add_button.on_click(handle_add)
save_button.on_click(handle_save)

refresh_dropdown()
if annotation_selector.options:
    populate_fields(annotation_selector.value)
draw_overlay()

toolbar = widgets.HBox([annotation_selector, class_selector, user_field])
bbox_inputs = widgets.HBox([x1_field, y1_field, x2_field, y2_field])
buttons = widgets.HBox([apply_button, delete_button, add_button, save_button])

display(widgets.VBox([toolbar, bbox_inputs, note_field, general_comment, buttons, status_output, overlay_output]))

In [7]:
import torch

cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")

if cuda_available:
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i} Name: {torch.cuda.get_device_name(i)}")

CUDA Available: True
Number of GPUs available: 1
GPU 0 Name: NVIDIA GeForce MX330


In [8]:
import subprocess
import os

image_id = SESSION_STATE.get("image_id")
if not image_id:
    raise RuntimeError("Run the annotation cells before launching fine-tuning.")

cmd = [
    sys.executable,
    "-m",
    "phase3_fault_type.incremental_finetune",
    "--image-id", image_id,
    "--epochs", "4",
    "--batch", "2",
    "--imgsz", "640",
    "--feedback-replay", "4",
    "--original-replay", "8",
    "--device", "0",
    "--link-strategy", "hardlink",
]

print("Running:", " ".join(cmd))

# Ensure the package import path resolves when called as a module from the notebook
env = os.environ.copy()
env["PYTHONPATH"] = str(REPO_ROOT) + os.pathsep + env.get("PYTHONPATH", "")

# Stream stdout live for progress visibility; also capture it to parse the RUN_LOG line
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    encoding="utf-8",
    errors="replace",
    env=env
)
run_log_path = None
while True:
    line = process.stdout.readline()
    if not line and process.poll() is not None:
        break
    if line:
        print(line, end="")
        if line.strip().startswith("RUN_LOG:"):
            # Format: RUN_LOG: <path>
            try:
                run_log_path = line.split("RUN_LOG:", 1)[1].strip()
            except Exception:
                pass

ret = process.wait()
if ret != 0:
    raise RuntimeError(f"Fine-tuning failed with exit code {ret}")

if run_log_path:
    print(f"\nRun log: {run_log_path}")
else:
    print("\nRun log path not reported. Check runs/logs/ for the latest log file.")

Running: d:\7th_sem\Software Design Project\transformer-maintenance-system\ano\Scripts\python.exe -m phase3_fault_type.incremental_finetune --image-id T5_faulty_001 --epochs 4 --batch 2 --imgsz 640 --feedback-replay 4 --original-replay 8 --device 0 --link-strategy hardlink
2025-10-20 00:31:59,083 | WARNING | CUDA not available (torch.cuda.is_available() is False); using CPU instead of '0'.
2025-10-20 00:31:59,083 | WARNING | Device override: requested '0', using 'cpu' instead.
2025-10-20 00:31:59,083 | INFO | Starting incremental fine-tune
2025-10-20 00:31:59,083 | INFO | RUN_NAME: finetune_T5_faulty_001_20251019T190158Z
2025-10-20 00:31:59,083 | INFO | RUN_LOG: D:\7th_sem\Software Design Project\transformer-maintenance-system\phase3_fault_type\runs\logs\finetune_T5_faulty_001_20251019T190158Z.log
2025-10-20 00:31:59,083 | INFO | Base weights: D:\7th_sem\Software Design Project\transformer-maintenance-system\phase2_fault_type\weights\best.pt
2025-10-20 00:31:59,083 | INFO | Image ID: T

In [9]:
history_path = config.RUN_HISTORY_PATH
if history_path.exists():
    entries = [json.loads(line) for line in history_path.read_text(encoding="utf-8").splitlines() if line.strip()]
    if entries:
        last_entry = entries[-1]
        print("Latest fine-tune run:")
        display(last_entry)
    else:
        print("History file is empty.")
else:
    print("No fine-tune runs recorded yet.")

Latest fine-tune run:


{'run_name': 'finetune_T5_faulty_001_20251019T190158Z',
 'timestamp': '20251019T190158Z',
 'image_id': 'T5_faulty_001',
 'transformer_id': 'T5',
 'weights_path': 'D:\\7th_sem\\Software Design Project\\transformer-maintenance-system\\phase3_fault_type\\weights\\finetune_T5_faulty_001_20251019T190158Z\\weights\\best.pt',
 'train_samples': 10,
 'feedback_samples': ['D:\\7th_sem\\Software Design Project\\transformer-maintenance-system\\Sample_Thermal_Images\\T5\\faulty\\T5_faulty_001.png',
  'D:\\7th_sem\\Software Design Project\\transformer-maintenance-system\\Sample_Thermal_Images\\T6\\faulty\\T6_faulty_002.jpg'],
 'original_samples': ['D:\\7th_sem\\Software Design Project\\transformer-maintenance-system\\Annotated_dataset\\train\\images\\T12_normal_003_jpg.rf.af6d0b03a1c58af45f5dd3510f616555.jpg',
  'D:\\7th_sem\\Software Design Project\\transformer-maintenance-system\\Annotated_dataset\\train\\images\\T1_faulty_031_jpg.rf.c5bcc3f9ee7743f20d4808c41f0eba4f.jpg',
  'D:\\7th_sem\\Software 

In [10]:
# Fine-tune summary: metrics and deltas vs previous run

from pathlib import Path

import json

from typing import Any, Dict, Optional

from phase3_fault_type import config



def _flatten_metrics(prefix: str, obj: Dict[str, Any], out: Dict[str, float]) -> None:

    if not isinstance(obj, dict):

        return

    for k, v in obj.items():

        key = f"{prefix}.{k}" if prefix else k

        if isinstance(v, dict):

            _flatten_metrics(key, v, out)

        elif isinstance(v, (int, float)):

            out[key] = float(v)



def _find_path_with_keyword(obj: Any, keyword: str) -> Optional[str]:

    try:

        if isinstance(obj, dict):

            for k, v in obj.items():

                if keyword in str(k).lower() and isinstance(v, str):

                    return v

                found = _find_path_with_keyword(v, keyword)

                if found:

                    return found

        elif isinstance(obj, list):

            for it in obj:

                found = _find_path_with_keyword(it, keyword)

                if found:

                    return found

    except Exception:

        pass

    return None



history_path = config.RUN_HISTORY_PATH

if not history_path.exists():

    print("No fine-tune runs recorded yet (history file missing).")

else:

    lines = [ln for ln in history_path.read_text(encoding="utf-8").splitlines() if ln.strip()]

    if not lines:

        print("History file is empty.")

    else:

        entries = []

        for ln in lines:

            try:

                entries.append(json.loads(ln))

            except Exception:

                # skip malformed lines but continue summarizing others

                continue

        if not entries:

            print("No valid run entries to summarize.")

        else:

            latest = entries[-1]

            prev = entries[-2] if len(entries) >= 2 else None



            # Flatten numeric metrics for consistent delta display

            latest_metrics_raw = latest.get("metrics") or {}

            latest_metrics: Dict[str, float] = {}

            _flatten_metrics("", latest_metrics_raw, latest_metrics)



            prev_metrics: Dict[str, float] = {}

            if prev is not None:

                _flatten_metrics("", prev.get("metrics") or {}, prev_metrics)



            print("=== Fine-tune Summary ===")

            print("Run:", latest.get("run_name") or latest.get("name") or "(unknown)")

            print("When:", latest.get("timestamp") or latest.get("time") or latest.get("started_at") or "")

            # Show brief training composition if available

            train_info = latest.get("train_pairs") or latest.get("train_size") or latest.get("dataset_info")

            if train_info:

                print("Train set:", train_info)



            # Ordered display of common YOLO metrics first, if present

            preferred_keys = [

                "map50-95", "map", "map50", "map75", "precision", "recall",

                "metrics.box.map50-95", "metrics.box.map", "metrics.box.map50", "metrics.box.map75",

                "metrics.precision", "metrics.recall"

            ]

            shown = set()

            def show_metric(key: str):

                if key in latest_metrics and key not in shown:

                    val = latest_metrics[key]

                    delta = None if key not in prev_metrics else val - prev_metrics[key]

                    if delta is None:

                        print(f" - {key}: {val:.4g}")

                    else:

                        sign = "+" if delta >= 0 else ""

                        print(f" - {key}: {val:.4g} ({sign}{delta:.4g})")

                    shown.add(key)



            for k in preferred_keys:

                show_metric(k)



            # Show any remaining numeric metrics

            remaining_keys = [k for k in sorted(latest_metrics.keys()) if k not in shown]

            if remaining_keys:

                print("Other metrics:")

                for k in remaining_keys:

                    val = latest_metrics[k]

                    delta = None if k not in prev_metrics else val - prev_metrics[k]

                    if delta is None:

                        print(f" - {k}: {val:.4g}")

                    else:

                        sign = "+" if delta >= 0 else ""

                        print(f" - {k}: {val:.4g} ({sign}{delta:.4g})")



            # Locate artifacts: weights and log

            weights_hint = _find_path_with_keyword(latest, "weight") or _find_path_with_keyword(latest, "model")

            log_hint = _find_path_with_keyword(latest, "log")

            if not log_hint:

                logs_dir = config.RUNS_DIR / "logs"

                try:

                    if logs_dir.exists():

                        latest_log = max(logs_dir.glob("*.log"), key=lambda p: p.stat().st_mtime, default=None)

                        if latest_log:

                            log_hint = str(latest_log)

                except Exception:

                    pass

            if weights_hint:

                print("Weights:", weights_hint)

            if log_hint:

                print("Log:", log_hint)


=== Fine-tune Summary ===
Run: finetune_T5_faulty_001_20251019T190158Z
When: 20251019T190158Z
Other metrics:
 - fitness: 0.2522
 - metrics/mAP50(B): 0.4981
 - metrics/mAP50-95(B): 0.2522
 - metrics/precision(B): 0.2707
 - metrics/recall(B): 0.6296
Weights: D:\7th_sem\Software Design Project\transformer-maintenance-system\phase3_fault_type\weights\finetune_T5_faulty_001_20251019T190158Z\weights\best.pt
Log: D:\7th_sem\Software Design Project\transformer-maintenance-system\phase3_fault_type\runs\logs\finetune_T5_faulty_001_20251019T190158Z.log


In [11]:
actions_export = SESSION_STATE["store"].export_actions(fmt="csv")
final_export = SESSION_STATE["store"].export_final_states(fmt="json")
print(f"Actions log exported to: {actions_export}")
print(f"Final annotations exported to: {final_export}")

Actions log exported to: D:\7th_sem\Software Design Project\transformer-maintenance-system\phase3_fault_type\annotations\exports\actions_20251019T190445Z.csv
Final annotations exported to: D:\7th_sem\Software Design Project\transformer-maintenance-system\phase3_fault_type\annotations\exports\final_states_20251019T190445Z.json
